# chinese graph

In [111]:
# coding: utf-8

import codecs
import networkx as nx
import numpy as np

In [128]:
f = codecs.open('./chinesegraph/chinese_words.csv', 'rb', encoding='utf-8')
words = [line.strip() for line in f]\
print 'number of words : ', len(words)

In [130]:
chars = set()
for w in words:
    chars = chars.union(set( w ))
    # Creating a little 'index'
print 'number of chars : ', len(chars)        

In [134]:
G = nx.Graph()
for ch in chars:
    G.add_node(ch,{'chinese':ch})

print 'number of chars nodes : ', G.number_of_nodes()

In [137]:
for i,w in enumerate(words):
    G.add_node(str(i),{'chinese': w })
    for ch in w:
        G.add_edge(str(i),ch)

print 'number of nodes (chars + words): ', G.number_of_nodes()

nx.write_graphml(G,'chinese_graph.graphml')

# word embedding

In [190]:
import networkx as nx
import numpy as np
import scipy as sp
import time
from random import choice
from math import log
import codecs

In [320]:
IS_CHINESE_TEST = True

if IS_CHINESE_TEST:
    G = nx.read_graphml('./data/graphs/chinese_graph.graphml', unicode) # Chinese graph
else:
    G = nx.read_graphml('graphs/bipartite_graph.graphml', unicode) # Korean language graph    

In [321]:
gen = nx.connected_component_subgraphs(G)
G = next(gen)
output_dir = "tmpDistancesFull2"

In [322]:
sts = nx.bipartite.sets(G)

In [323]:
words = None
hanjas = None
wordSet = None

if len(sts[0]) > 6000: 
    words = list(sts[0])
    wordSet = sts[0]
    hanjas = list(sts[1])
else: 
    words = list(sts[1])
    wordSet = sts[1]
    hanjas = list(sts[0])


In [324]:
wordic = {G.node[e]['chinese']:k for k,e in enumerate(words)}

In [325]:
synonyms = None

In [326]:
with codecs.open('./data/synonyms/all_chinese_synonyms.csv', 'r', encoding='utf-8') as f:
        synonyms = [line.strip() for line in f]

In [327]:
synDP = []
randDP = []


In [328]:
A = nx.bipartite.biadjacency_matrix(G, hanjas)
A = A.astype(float)

In [329]:
weight = False # In case we chose not to do TF-IDF weighting

In [330]:
minK = 666
maxK = 668

In [331]:
U,s,V = sp.sparse.linalg.svds(A,800)
Vtr = V.transpose()

In [316]:
for pr in synonyms:

    word = pr.split(',')
    c1 = word[0]
    c2 = word[1]
    
    if c1 not in wordic or c2 not in wordic: continue

    v1 = Vtr[wordic[c1]]
    v2 = Vtr[wordic[c2]]
    synDP.append([ np.dot(v1,v2), c1.encode('utf-8'), c2.encode('utf-8') ] )

In [317]:
for _ in synonyms:
    v1 = choice(Vtr)
    v2 = choice(Vtr)
    randDP.append(np.dot(v1,v2))

In [318]:
output_dir = './output'
with codecs.open("{}/randdist_k={}.csv".format(output_dir,k), 'w', encoding='utf-8') as f:
    wr = csv.writer(f)
    wr.writerows([[r] for r in randDP])

In [319]:
with codecs.open("{}/syndist_k={}.csv".format(output_dir,k), 'w') as f:
    wr = csv.writer(f)
    wr.writerows(synDP)

# scrap iciba

In [661]:
from bs4 import BeautifulSoup
import time
from itertools import combinations
from selenium import webdriver
import urllib

In [662]:
G = nx.read_graphml('data/graphs/chinese_graph.graphml', unicode)

In [663]:
words = list(G.node[a]['chinese'] for a in G.node.keys() if 'chinese' in G.node[a])

In [664]:
base_url = 'http://www.iciba.com/'

In [665]:
allComb = []
allSyn = []
allAnt = []


In [666]:
# for i, ch in enumerate(words):
#     if len(ch) is 2:
#         print i, ch
# 46

w = words[46]

In [667]:
print w

负担


In [668]:
query = base_url + w
query

u'http://www.iciba.com/\u8d1f\u62c5'

In [686]:
r = urllib.urlopen(query.encode('utf-8')).read()

In [687]:
soup = BeautifulSoup(res.text)

In [688]:
len( soup.prettify())

56737

In [690]:
soup.select(".opposite-word")

[]

In [658]:
print soup.prettify().find( ('休伦特').decode('utf-8') )

-1


In [659]:
print soup.prettify()[:300]

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8">
   <meta content="IE=Edge" http-equiv="X-UA-Compatible">
    <meta content="cU06Y0SNXq" name="baidu-site-verification"/>
    <title>
     负担的英文_负担的英语翻译_负担用英语怎么说_爱词霸在线词典
    </title>
    <meta content="负担,负担的英文,负担英语,负担的翻译,负担用英语怎么说,怎么用英语翻译负担" nam


In [680]:
query

u'http://www.iciba.com/\u8d1f\u62c5'

In [673]:
from lxml import html
import requests

In [674]:
page = requests.get(query)

In [676]:
tree = html.fromstring(page.content)

In [450]:
time.sleep(0.2)


res = r.get(baseUrl+w)

In [452]:
sp = BeautifulSoup(res.text)
ows = sp.select('.opposite-word')

In [417]:
query_url = "http://www.chinesetools.eu/tools/synonym/#?q=" + w + "&Submit=Search"
synPairs = []
antPairs = []

In [426]:
res = r.get(query_url)
sp = BeautifulSoup(res.text)

In [427]:
ows = sp.select('synonyms')
ows

[]

[]

In [408]:
res

<Response [404]>

In [374]:
for i,w in enumerate(wds):
    if i % 20 == 0: 
        print("Done {} words. Got {} synonyms, {} antonyms."
              .format(i,len(synPairs),len(antPairs)))
    time.sleep(0.2)
    res = r.get(baseUrl+w)
    sp = BeautifulSoup(res.text)
    ows = sp.select('.opposite-word')
    if len(ows) == 0: continue
    
    art = ows[0].parent
    dvs = art.select('div')
    doneSyn = False
    
    break
    
    for dv in dvs[1:]: # Skipping the first one...
        if dv['class'] == 'opposite-word':
            doneSyn = True
            continue
        txt = dv.select('a')[0].text.strip()
        if not doneSyn: 
            synPairs.append([w,txt])
        else: 
            antPairs.append([w,txt])

Done 0 words. Got 0 synonyms, 0 antonyms.
Done 20 words. Got 0 synonyms, 0 antonyms.
Done 40 words. Got 0 synonyms, 0 antonyms.
Done 60 words. Got 0 synonyms, 0 antonyms.
Done 80 words. Got 0 synonyms, 0 antonyms.
Done 100 words. Got 0 synonyms, 0 antonyms.
Done 120 words. Got 0 synonyms, 0 antonyms.
Done 140 words. Got 0 synonyms, 0 antonyms.
Done 160 words. Got 0 synonyms, 0 antonyms.
Done 180 words. Got 0 synonyms, 0 antonyms.


KeyboardInterrupt: 